# CanYouCatchIt?
A web application allowing you to obtain the percentage of chance that your bus/tram/metro is late. 💻🤖🎲🚌 🚎🚇🔮

_Build with the STIB API (available [here](https://opendata.stib-mivb.be/store/))_

# Notes: Making some models 💻🤖🚌 🚎🚇
We are here to explore the data

## Load the data

Write a function loading the csv files

In [ ]:
# import
import glob
import pandas as pd
import seaborn as sns
import os

# Set the path to the directory holding CSV files
DELAY_PATH = '/home/haeresis/Documents/Github/CanYouCatchIt/machine_learning/data'

def load_delay_data(delay_path=DELAY_PATH):
    """
    Load the cvs file in a panda dataframe
    """
    return pd.concat([pd.read_csv(f) for f in glob.glob('../data/delay*.csv')], ignore_index = True)

## Looking For Correlations

In [ ]:
corr_matrix = delay.corr()

Look at how much each attribute correlates with the delay value

In [ ]:
corr_matrix["delay"].sort_values(ascending=False) # warning: this check only linear correlation

In [ ]:
plt.figure(figsize= (10,10), dpi=100)
sns.heatmap(corr_matrix)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["delay", "humidity", "hour", "wind", "temp"]
scatter_matrix(delay[attributes], figsize=(12, 8))

In [ ]:
delay.plot(kind="scatter", x="hour", y="delay", alpha=0.1)

Try adding a new feature combining 3 features (hour, minute, seconds)

In [ ]:
delay["hour_and_minute"] = delay["hour"]*3600 + delay["minute"]*60
delay["hour_minute_and_seconds"] = delay["hour"]*3600 + delay["minute"]*60 + delay["seconds"]

In [ ]:
delay.head(3)

In [ ]:
corr_matrix = delay.corr()
corr_matrix["delay"].sort_values(ascending=False)

In [ ]:
delay.plot(kind="scatter", x="hour_and_minute", y="delay", alpha=0.2)

In [ ]:
delay.plot(kind="scatter", x="hour_minute_and_seconds", y="delay", alpha=0.2)

With the experiment we can see that the attributes that are the most corroleted (linearly) with the delay are the temperature, the wind and the humidity.
We can also see that the hour and the minute attributes are not linearly corroleted with the delay